# Debug for spin-up prediction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import torch
import argparse
from pathlib import Path
import pickle
import sys
sys.path.append('../')
import time
from src.utils.score import *
from src.utils.plot import plot_iter_result, plot_increment
from collections import OrderedDict
from src.inference.autoregressive_inference import autoregressive_inference_4dvar, autoregressive_inference_cvae, autoregressive_inference_after_spin
from src.inference.spinup_pred_inference import spinup_pred_loop_inference
from src.models.fourcast_module import FourCastNetLitModule # 预报模型
from src.models.cvae_module import CVAELitModule
from src.utils.utils import str2bool

## The pipeline of the spinup_pred_loop

In [2]:
# def spinup_pred_loop_inference(data_dir, 
#                                xb_dir, 
#                                obs_dir, 
#                                obs_mask_dir, 
#                                pretrain_dir, 
#                                damodel_dir, 
#                                output_dir, 
#                                init_time, 
#                                dtmodel, 
#                                dt_obs,
#                                dt_da_pred, 
#                                daw, 
#                                obs_partial, 
#                                obs_single, 
#                                obserr_level, 
#                                spinup_length,
#                                prediction_length, 
#                                decorrelation_time, 
#                                out_iter, 
#                                mode, 
#                                da_method, 
#                                device):
#     # 读取均值、方差、经纬度网格数据
#     with open(Path(f'{data_dir}')/f'scaler.pkl', 'rb') as f:
#         item = pickle.load(f)
#         lon = item['lon']
#         lat = item['lat']
#         mean = item['mean']
#         std = item['std']
#         f.close()
        
#     # 读取真值
#     valid = xr.open_mfdataset(f'{obs_dir}/{mode}/*.nc', combine='by_coords')
#     clim = torch.from_numpy(valid.mean('time')['z'].values).to(device, dtype=torch.float)
#     # valid = xr.merge([z500_valid])
#     # 读取观测数据
#     obs = xr.open_mfdataset(f'{obs_dir}/{mode}/observations_*_err{obserr_level}.nc', combine='by_coords')
#     obs = (obs - mean) / std
#     obs_masks = xr.open_mfdataset(f'{obs_mask_dir}/{mode}/mask_observations_single_*.nc' if obs_single else f'{obs_mask_dir}/{mode}/mask_observations_{int(100*obs_partial)}_*.nc', combine='by_coords')
#     obs_masks = obs_masks.sel(time=obs['time'])['z'].values
#     # 选取有观测对应时刻的真值
#     valid = valid.sel(time=obs['time'])

#     start_id = int((obs['time'][0]-valid["time"][0]).values.astype('timedelta64[h]') / np.timedelta64(1, 'h'))
#     # 读取背景场数据
#     xbs = xr.open_mfdataset(f'{xb_dir}/{mode}/background_predlen120_dtmodel6_year*.nc', combine='by_coords')
#     if da_method == '4DVar':
#         # NMC方法计算B矩阵，并对B矩阵求逆
#         pred_24 = xbs.sel(init_time=-24)
#         pred_48 = xbs.sel(init_time=-48)
#         diff = pred_48 - pred_24
#         diff_value = np.reshape(diff['z'].values, [diff['z'].values.shape[0], -1])
#         B = np.cov(diff_value.T) / diff_value.shape[-1]
#         B_inv = np.linalg.inv(B)
#     xbs = (xbs - mean) / std
#     # 计算R矩阵并求逆
#     noise = obserr_level * mean / std
#     obs_value = obs['z'].values
#     N = obs_value.shape[-1]*obs_value.shape[-2]
#     R_inv = 1/(noise ** 2) * np.eye(N, N) # 观测误差协方差求逆
    
#     # 提取用于作为初始场的xb
#     xbs = xbs.sel(init_time=-init_time)
    
#     # 加载预训练预报模型
#     # 用于做预报的模型
#     module = FourCastNetLitModule.load_from_checkpoint(f'{pretrain_dir}/nv_fourcastnet_leadtime{dtmodel}h.ckpt')
#     prednet = module.net.to(device).eval()
#     # 用于4DVar做同化的预报模型
#     if da_method == '4DVar':
#         da_module = FourCastNetLitModule.load_from_checkpoint(f'{pretrain_dir}/nv_fourcastnet_leadtime{dt_da_pred}h.ckpt')
#         da_prednet = da_module.net.to(device).eval()
#     elif da_method == 'CVAE':
#         da_module = CVAELitModule.load_from_checkpoint(f'{damodel_dir}/cvae_obs_single.ckpt' if obs_single else f'{damodel_dir}/cvae_obs{int(100*obs_partial)}.ckpt')
#         da_model = da_module.net.to(device).eval()
#     # 取初始场
#     spinup_length = 24 * spinup_length
#     prediction_length = 24 * prediction_length
#     n_samples_per_year = len(valid['z'])
#     n_samples = n_samples_per_year - (spinup_length + prediction_length + dtmodel) // dt_obs
#     stop = n_samples
#     ics_spinup = np.arange(start_id, stop, decorrelation_time // dt_obs)
#     ics_pred = np.arange(start_id+spinup_length//dt_obs, stop+spinup_length//dt_obs, decorrelation_time // dt_obs)
    
#     fcs, xbs_ = [], []
#     val_rmse, val_acc, val_mae = [], [], []
#     da_times = []
#     for i, ic_sinup in enumerate(ics_spinup):
#         print(f'Exp {i+1}/{len(ics_spinup)} start.')
#         time4da = 0
#         if da_method == '4DVar':
#             fc_spin, xb_spin, rmse_spin, acc_spin, mae_spin, da_time = autoregressive_inference_4dvar(ic_sinup, 
#                                                                                                     start_id,
#                                                                                                     out_iter,
#                                                                                                     mean, 
#                                                                                                     std, 
#                                                                                                     valid,
#                                                                                                     xbs['z'].values[ic_sinup],
#                                                                                                     obs_value, 
#                                                                                                     prednet,
#                                                                                                     da_prednet,
#                                                                                                     dtmodel, 
#                                                                                                     dt_da_pred, 
#                                                                                                     daw,
#                                                                                                     dt_obs, 
#                                                                                                     B_inv, 
#                                                                                                     R_inv, 
#                                                                                                     spinup_length + dtmodel, 
#                                                                                                     obs_masks,
#                                                                                                     clim,
#                                                                                                     device)
#         elif da_method == 'CVAE':
#             fc_spin, xb_spin, rmse_spin, acc_spin, mae_spin, da_time = autoregressive_inference_cvae(ic_sinup, 
#                                                                                                         start_id,
#                                                                                                         mean, 
#                                                                                                         std, 
#                                                                                                         valid, 
#                                                                                                         xbs['z'].values[ic_sinup],
#                                                                                                         obs['z'].values, 
#                                                                                                         prednet,
#                                                                                                         da_model,
#                                                                                                         dtmodel, 
#                                                                                                         daw,
#                                                                                                         dt_obs, 
#                                                                                                         spinup_length + dtmodel, 
#                                                                                                         obs_masks, 
#                                                                                                         clim,
#                                                                                                         device)
#         time4da += da_time
#         print(f'Time for DA of {da_method} is {time4da}[s]!')
#         da_times.append(time4da)
#         ini_data = fc_spin.values[-1]
#         fc, rmse, acc, mae = autoregressive_inference_after_spin(ics_pred[i],
#                                                                   mean,
#                                                                   std,
#                                                                   valid,
#                                                                   ini_data,
#                                                                   prednet,
#                                                                   dtmodel,
#                                                                   dt_obs,
#                                                                   spinup_length,
#                                                                   prediction_length + dtmodel, 
#                                                                   clim,
#                                                                   device)
#         fc = xr.concat([fc_spin, fc], dim='lead_time')
#         rmse = np.concatenate([rmse_spin, rmse], axis=1)
#         acc = np.concatenate([acc_spin, acc], axis=1)
#         mae = np.concatenate([mae_spin, mae], axis=1)
#         fcs.append(fc)
#         xbs_.append(xb_spin)
#         val_rmse.append(rmse)
#         val_acc.append(acc)
#         val_mae.append(mae)
#         del fc, xb_spin, rmse, acc, mae, da_time
#         print(f'Exp {i+1}/{len(ics_spinup)} end.')
    
#     # 对每一组实验进行统计
#     fc_iter = xr.merge(fcs)
#     xb_iter = xr.merge(xbs_)
#     val_rmse = np.mean(np.concatenate(val_rmse, 0), axis=0)
#     val_acc = np.mean(np.concatenate(val_acc, 0), axis=0)
#     val_mae = np.mean(np.concatenate(val_mae, 0), axis=0)
#     # 保存RMSE
#     xr_rmse = [xr.DataArray(
#                         val_rmse[:,0],
#                         dims=['Lead Time'],
#                         coords={
#                             'Lead Time': fc_iter.lead_time.values,
#                         },
#                         name='z'
#                     )]
#     xr_rmse = xr.merge(xr_rmse)
#     # 保存ACC
#     xr_acc = [xr.DataArray(
#                         val_acc[:,0],
#                         dims=['Lead Time'],
#                         coords={
#                             'Lead Time': fc_iter.lead_time.values,
#                         },
#                         name='z'
#                     )]
#     xr_acc = xr.merge(xr_acc)
#     # 保存MAE
#     xr_mae = [xr.DataArray(
#                         val_mae[:,0],
#                         dims=['Lead Time'],
#                         coords={
#                             'Lead Time': fc_iter.lead_time.values,
#                         },
#                         name='z'
#                     )]
#     xr_mae = xr.merge(xr_mae)
#     # 保存分析预报循环中的背景场和预报结果
#     return fc_iter, xb_iter, xr_rmse, xr_acc, xr_mae, da_times

## Parameters for experiment

In [3]:
data_dir='/public/home/wangwuxing01/research/weatherbench/geopotential_500'
xb_dir='/public/home/wangwuxing01/research/weatherbench/da_data/background'
obs_dir='/public/home/wangwuxing01/research/weatherbench/da_data/observation'
obs_mask_dir='/public/home/wangwuxing01/research/weatherbench/da_data/observation_mask'
pretrain_dir='/public/home/wangwuxing01/research/weatherbench/weights/pretrain'
damodel_dir='/public/home/wangwuxing01/research/weatherbench/weights/cvae'
output_dir='.'
init_time=120
dtmodel=6
dt_obs=3
dt_da_pred=3
daw=12
obs_partial=0.5
obs_single=False
obserr_level=0.015
spinup_length=5
prediction_length=10
decorrelation_time=2400000
out_iter=5
mode='test'
da_method='CVAE'
device='cuda:0'

## Run experiments

In [4]:
fc_iter, xb_iters, xr_rmse, xr_acc, xr_mae = spinup_pred_loop_inference(data_dir, 
                                                                        xb_dir, 
                                                                        obs_dir, 
                                                                        obs_mask_dir, 
                                                                        pretrain_dir, 
                                                                        damodel_dir, 
                                                                        output_dir, 
                                                                        init_time, 
                                                                        dtmodel, 
                                                                        dt_obs, 
                                                                        dt_da_pred, 
                                                                        daw, 
                                                                        obs_partial, 
                                                                        obs_single, 
                                                                        obserr_level, 
                                                                        spinup_length,
                                                                        prediction_length, 
                                                                        decorrelation_time, 
                                                                        out_iter, 
                                                                        mode, 
                                                                        da_method, 
                                                                        device)

/public/home/wangwuxing01/.conda/envs/afnonet-core/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.
  rank_zero_warn(
/public/home/wangwuxing01/.conda/envs/afnonet-core/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:268: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


Exp 1/1 start.
Time for DA of CVAE is 0.34151411056518555[s]!
Exp 1/1 end.
<xarray.Dataset>
Dimensions:    (lead_time: 10, time: 1, lat: 32, lon: 64)
Coordinates:
  * lead_time  (lead_time) int64 12 24 36 48 60 72 84 96 108 120
  * time       (time) datetime64[ns] 2017-01-06
  * lat        (lat) float64 -87.19 -81.56 -75.94 -70.31 ... 75.94 81.56 87.19
  * lon        (lon) float64 0.0 5.625 11.25 16.88 ... 337.5 343.1 348.8 354.4
Data variables:
    z          (lead_time, time, lat, lon) float32 5.104e+04 ... 4.94e+04
./xb_iter_cvae_obserr0.015_obspartial0.5.nc saved!
./fc_iter_cvae_obserr0.015_obspartial0.5.nc saved!
./rmse_cvae_obserr0.015_obspartial0.5.nc saved!
./acc_cvae_obserr0.015_obspartial0.5.nc saved!
./mae_cvae_obserr0.015_obspartial0.5.nc saved!


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
fc_iter

## Analysis the results

In [ ]:
xr_rmse

In [22]:
xr_acc

<xarray.Dataset>
Dimensions:    (Lead Time: 54)
Coordinates:
  * Lead Time  (Lead Time) int64 0 6 12 18 24 30 36 ... 282 288 294 300 306 312
Data variables:
    z          (Lead Time) float32 0.534 0.5376 0.7963 ... 0.6246 0.5789 0.6124

In [ ]:
xr_rmse

In [ ]:
xr_acc